In [1]:
import sys, os, csv
import numpy as np
import pandas as pd
import timeit
import pickle
import theano
import theano.tensor as T

os.chdir(sys.path[0] + "\..")

import yaml

from theano import shared, function
from scipy import stats
from datetime import datetime as dt

from reslogit.models import Logit, ResNet, MLP
from reslogit.core import *
import reslogit.optimizers as optimizers

In [2]:
# constants
FLOATX = theano.config.floatX

# read data file from .csv
raw_data = pd.read_csv(os.getcwd() + '/data/data-20190702_2.csv')

# read configuration file
with open('config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
# keep track of time
config['timestamp'] = dt.now().strftime("%Y-%m-%d %H:%M:%S")

# defines the inputs and output
x_data = raw_data.iloc[:, 1:-1]
y_data = raw_data['mode']-1  # -1 for indexing at 0

# defines the list of explanatory variable names
config['variables'] = list(x_data.columns)

# number of observations
config['n_obs'] = raw_data.shape[0] 

# number of variables/choices
config['n_vars'] = x_data.shape[1]
config['n_choices'] = len(config['choices'])

# slicing index for train/valid split
slice = np.floor(0.7*config['n_obs']).astype(int)
config['slice'] = slice

# slices x and y datasets into train/valid
train_x_data, valid_x_data = x_data.iloc[:slice], x_data.iloc[slice:]
train_y_data, valid_y_data = y_data.iloc[:slice], y_data.iloc[slice:]

# load train/valid datasets into shared module
train_x_shared, train_y_shared = shared_dataset(train_x_data, train_y_data)
valid_x_shared, valid_y_shared = shared_dataset(valid_x_data, valid_y_data)

# number of train/valid batches
n_train_batches = train_y_data.shape[0] // config['batch_size']
n_valid_batches = valid_y_data.shape[0] // config['batch_size']
config['n_train_batches'] = n_train_batches
config['n_valid_batches'] = n_valid_batches

# Theano tensor variables
idx = T.lscalar()  # index to [mini]batch
x = T.matrix('x')
y = T.ivector('y')

In [3]:
if config['model_type'] == 'ResNet':
    # create ResNet model
    model = ResNet(
        input=x, choice=y, 
        n_vars=config['n_vars'], 
        n_choices=config['n_choices'], 
        n_layers=config['n_layers']
    )
    cost = model.negative_log_likelihood(y)
    opt = optimizers.RMSProp(model.params)
    updates = opt.run_update(
        cost, model.params, 
        masks=model.params_mask, learning_rate=config['learning_rate']
    )
elif config['model_type'] == 'MLP':
    # create MLP model
    model = MLP(
        input=x, choice=y, 
        n_vars=config['n_vars'], 
        n_choices=config['n_choices'], 
        n_layers=config['n_layers']
    )
    cost = model.negative_log_likelihood(y)
    opt = optimizers.RMSProp(model.params)
    updates = opt.run_update(
        cost, model.params, 
        learning_rate=config['learning_rate']
    )
elif config['model_type'] == 'MNL':
    # create MNL model
    config['n_layers'] = 0
    model = Logit(
        input=x, choice=y, 
        n_vars=config['n_vars'], 
        n_choices=config['n_choices']
    )
    cost = model.negative_log_likelihood(y)
    opt = optimizers.RMSProp(model.params)
    updates = opt.run_update(
        cost, model.params, 
        masks=model.params_mask, learning_rate=config['learning_rate']
    )
print(config['model_type'])

MNL


In [4]:
batch_size = config['batch_size']

model.train_model = function(
    inputs=[idx],
    outputs=cost,
    updates=updates,
    allow_input_downcast=True,
    givens={
        x: train_x_shared[idx * batch_size: (idx + 1) * batch_size],
        y: train_y_shared[idx * batch_size: (idx + 1) * batch_size],
    },
)

model.validate_model = function(
    inputs=[],
    outputs=cost,
    allow_input_downcast=True,
    givens={
        x: valid_x_shared,
        y: valid_y_shared,
    },
)

model.predict_model = function(
    inputs=[],
    outputs=model.errors(y),
    allow_input_downcast=True,
    givens={
        x: valid_x_shared,
        y: valid_y_shared,
    },
)
model.train_loss_model = function(
    inputs=[],
    outputs=model.errors(y),
    allow_input_downcast=True,
    givens={
        x: train_x_shared,
        y: train_y_shared,
    },
)


model.hessians = function(
    inputs=[idx],
    outputs=model.get_gessians(y),
    allow_input_downcast=True,
    givens={
        x: train_x_shared[idx * batch_size: (idx + 1) * batch_size],
        y: train_y_shared[idx * batch_size: (idx + 1) * batch_size],
    },
)

model.derivatives = function(
    inputs=[idx],
    outputs=model.get_derivatives(y),
    allow_input_downcast=True,
    givens={
        x: train_x_shared[idx * batch_size: (idx + 1) * batch_size],
        y: train_y_shared[idx * batch_size: (idx + 1) * batch_size],
    },
)

In [5]:
valid_freq = min(200, n_train_batches)
best_validation_ll = np.inf
start_time = timeit.default_timer()
done_looping = False
epoch = 0
step = 0

filename = '{}{}_bestmodel.pkl'.format(config['model_type'], config['n_layers'])
training_frame = pd.DataFrame(
    columns=['epoch', 'minibatch', 'batches', 'train_ll', 'valid_ll', 'valid_err', 'train_err']
)

while (epoch < config['n_epochs']) and (not done_looping):
    epoch = epoch + 1
    training_ll = 0
    for i in range(n_train_batches):
        # accumulating average score
        minibatch_ll = model.train_model(i)
        training_ll = (training_ll * i + minibatch_ll)/(i + 1)
        
        iteration = (epoch - 1) * n_train_batches + i

        if (iteration + 1) % valid_freq == 0:
            validation_ll = np.sum(model.validate_model())
            #################################
            # track and save training stats #
            #################################
            training_step = {
                'epoch': epoch, 
                'minibatch': i + 1, 
                'batches': n_train_batches, 
                'train_ll': training_ll * n_train_batches, 
                'valid_ll': validation_ll, 
                'valid_err': None,
                'train_err': None,
            }
            training_frame.loc[step] = training_step
            #################################

            # check prediction accuracy
            error = np.mean(model.predict_model())
            training_frame.loc[step, 'valid_err'] = error

            
            
            if validation_ll < best_validation_ll:
                print(('epoch {:d}, minibatch {:d}/{:d}, '
                       'validation likelihood {:.2f}').format(
                        epoch, i + 1, n_train_batches, validation_ll))

                # improve patience if loss improvement is good enough
                if validation_ll < best_validation_ll * config['improvement_threshold']:
                    config['patience'] = max(config['patience'], iteration * config['patience_increase'])

                # keep track of best validation score
                best_validation_ll = validation_ll

                best_error = np.mean(model.predict_model())
                training_frame.loc[step, 'valid_err'] = best_error
                print('validation error  {:.2%}'.format(best_error))

                # save the best model
                with open(filename, 'wb') as f:
                    pickle.dump([model, config], f)

            step = step + 1

        if epoch > 200:
            done_looping = True
            break

end_time = timeit.default_timer()
run_time = end_time - start_time

#############################
# parameter extraction step #
#############################
print('processing model statistics...')
with open(filename, 'rb') as f:
    model, config = pickle.load(f)

# format model beta params and ASCs
model_stat = {}
if config['model_type'] in ['MNL', 'ResNet']:
    beta = model.beta.eval().round(3)
    beta_df = pd.DataFrame(beta, index=config['variables'], columns=config['choices'])
    model_stat['beta_params'] = beta_df

    asc = model.params[1].eval().round(3)
    asc_df = pd.DataFrame(asc.reshape((1,-1)), index=['ASC'], columns=config['choices'])
    model_stat['asc_params'] = asc_df
    
    # compute sandwich estimator "middle"
    B = np.mean([model.derivatives(i) for i in range(n_train_batches)], axis=0)
    B = B[0]
    model_stat['sandwich_B'] = B
    
    # compute sandwich estimator "bread"
    neg_A = np.mean([model.hessians(i) for i in range(n_train_batches)], axis=0)
    invneg_A = 1/np.diag(neg_A[0])
    model_stat['sandwich_invneg_A'] = invneg_A
    
    # compute std. err and t-stat
    h = np.mean([model.hessians(i) for i in range(n_train_batches)], axis=0)
    model_stat['beta_stderr'] = pd.DataFrame(
        np.sqrt(1/np.diag(h[0]).reshape((config['n_vars'], config['n_choices'])))/(batch_size-1), 
        index=config['variables'], 
        columns=config['choices']
    )
    model_stat['beta_t_stat'] = pd.DataFrame(
        beta / model_stat['beta_stderr'], index=config['variables'], columns=config['choices'])
    
# format ResNet residual matrix
if config['model_type'] == 'ResNet':
    model_stat['residual_matrix'] = []
    for l in range(config['n_layers']):
        # create a pandas correlation matrix table
        mat = model.resnet_layers[l].params[0].eval().round(2)
        df = pd.DataFrame(
            data=mat, index=config['choices'], columns=config['choices'])
        model_stat['residual_matrix'].append(df)

# misc: runtime and training curves
model_stat['run_time'] = str(np.round(run_time / 60., 3))+' minutes'
model_stat['training_frame'] = training_frame

# re-save model, configuration and statistics     
with open(filename, 'wb') as f:
    pickle.dump([model, config, model_stat], f)        
#############################

# print final verbose output
print(('Optimization complete with best validation likelihood of {:.2f}, '
       'and validation error of {:.2%}').format(best_validation_ll, best_error))
print(('The code run for {:d} epochs, with {:.2f} epochs/sec').format(
        epoch, 1. * epoch / run_time))
print('training complete')

epoch 1, minibatch 200/1320, validation likelihood 21768.39
validation error  41.76%
epoch 1, minibatch 400/1320, validation likelihood 20743.21
validation error  41.77%
epoch 1, minibatch 600/1320, validation likelihood 20171.09
validation error  41.65%
epoch 1, minibatch 800/1320, validation likelihood 19611.28
validation error  38.54%
epoch 1, minibatch 1200/1320, validation likelihood 18689.50
validation error  38.58%
epoch 2, minibatch 80/1320, validation likelihood 18330.98
validation error  34.93%
epoch 2, minibatch 280/1320, validation likelihood 18110.92
validation error  36.36%
epoch 2, minibatch 480/1320, validation likelihood 17846.46
validation error  35.64%
epoch 2, minibatch 680/1320, validation likelihood 17844.19
validation error  35.78%
epoch 2, minibatch 1080/1320, validation likelihood 17592.13
validation error  33.04%
epoch 2, minibatch 1280/1320, validation likelihood 17355.43
validation error  31.99%
epoch 3, minibatch 160/1320, validation likelihood 17290.01
val

C:\Users\Melvin\AppData\Local\miniconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


Optimization complete with best validation likelihood of 15873.71, and validation error of 27.87%
The code run for 201 epochs, with 6.07 epochs/sec
training complete


In [6]:
# if config['model_type'] == 'ResNet': 
#     print(model_stat['residual_matrix'])
# MNL Optimization complete with best validation likelihood of 15873.71, and validation error of 27.93% $$
# MLP-2 Optimization complete with best validation likelihood of 15643.12, and validation error of 27.56% $$
# MLP-4 Optimization complete with best validation likelihood of 17406.30, and validation error of 29.74% $$
# MLP-8 Optimization complete with best validation likelihood of 17370.73, and validation error of 29.70% $$
# MLP-16 Optimization complete with best validation likelihood of 17403.81, and validation error of 29.69% $$

# ResNet-2 Optimization complete with best validation likelihood of 13443.44, and validation error of 23.87% $$
# ResNet-4 Optimization complete with best validation likelihood of 13082.63, and validation error of 23.58% $$
# ResNet-8 Optimization complete with best validation likelihood of 12894.89, and validation error of 23.33% $$
# ResNet-16 Optimization complete with best validation likelihood of 12938.32, and validation error of 23.30% $$

In [7]:
filename = 'MNL0_bestmodel.pkl'

with open(filename, 'rb') as f:
    data = pickle.load(f)
    
model, config, model_stat = data


In [8]:
model_stat['beta_params'].stack().reset_index().iloc[[0,1,2,10,11,15,18,28,36,70,72,73,77,79,80,84,87,91,94,133,134,135,140,142,144,148]][0]

0     -0.057
1     -0.990
2     -0.751
10    -0.841
11    -2.273
15    -0.854
18    -2.540
28     0.058
36    -1.271
70     0.354
72     0.297
73    -2.197
77    -0.627
79     0.870
80     0.863
84     0.988
87    -1.738
91    -1.357
94    -0.067
133   -0.119
134   -1.048
135    0.109
140   -0.055
142   -0.011
144   -1.853
148   -2.776
Name: 0, dtype: float64

In [9]:
model_stat['beta_stderr']

,auto,bike,transit,walk,auto+transit,other_mode,other_combi
weekend,0.036359,0.080725,0.042279,0.056549,0.208347,0.313384,0.066392
hour_8_10,0.035520,0.052861,0.036120,0.069821,0.121295,0.169593,0.072783
hour_11_13,0.041167,0.072678,0.046455,0.058997,0.216774,0.274668,0.084709
hour_14_16,0.032195,0.056218,0.035004,0.054733,0.087268,0.173706,0.076437
hour_17_19,0.028717,0.045191,0.029663,0.052161,0.068352,0.192770,0.068676
hour_20_22,0.048984,0.092328,0.051690,0.086596,0.167335,0.353132,0.097496
hour_23_1,0.086568,0.165032,0.094337,0.193433,0.409996,0.285384,0.140163
hour_2_4,0.224390,0.599124,0.321459,0.458743,0.965251,0.410940,0.335465
hour_5_7,0.043677,0.083513,0.046041,0.117259,0.108250,0.521196,0.104030
num_coord,0.003407,0.006010,0.003866,0.008390,0.009068,0.018345,0.008398


In [10]:
b = model_stat["sandwich_B"]
a = model_stat["sandwich_invneg_A"]
c = np.sqrt(a * (1/b * 1/b) * a)

rob_std_err = pd.DataFrame(
        c.reshape((config['n_vars'], config['n_choices']))/(batch_size-1), 
        index=config['variables'], 
        columns=config['choices']
    )
rob_std_err

,auto,bike,transit,walk,auto+transit,other_mode,other_combi
weekend,0.386267,7.335337,1.569170,2.314049,23.382455,6.002546e+01,0.834472
hour_8_10,0.363625,7.904245,0.310207,7.985548,15.004514,6.317744e+01,5.538007
hour_11_13,0.461933,47.886171,0.945079,4.457516,48.866362,9.748339e+01,2.928527
hour_14_16,0.275542,37.837031,0.330756,4.697754,5.959110,6.497259e+01,5.417648
hour_17_19,0.185915,6.478938,0.155656,6.253785,3.481084,4.782675e+01,4.615361
hour_20_22,2.680789,16.936906,1.865111,79.020577,28.897268,1.827103e+02,6.243710
hour_23_1,46.798054,42.508332,22.486933,97.791181,332.267242,2.274595e+02,14.755628
hour_2_4,93.056687,1841.936248,711.239874,2374.287465,6926.861083,9.614551e+02,568.729877
hour_5_7,1.207914,21.068693,0.743020,61.515557,12.662558,7.442862e+02,18.942554
num_coord,0.000209,0.023326,0.000199,0.004713,0.003508,2.469074e-02,0.002071
